# Initialization

In [1]:
from torch.utils.data import DataLoader
from torchvision import transforms

import torchvision.models as models
import torch.optim as optim
import torch.nn as nn 
import torch

import torchvision

device = torch.device('mps')

BATCH_SIZE = 64
EPOCH = 50

/Users/bahk_insung/miniconda3/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/Users/bahk_insung/miniconda3/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN2at4_ops19empty_memory_format4callEN3c108ArrayRefIxEENS2_8optionalINS2_10ScalarTypeEEENS5_INS2_6LayoutEEENS5_INS2_6DeviceEEENS5_IbEENS5_INS2_12MemoryFormatEEE
  Referenced from: <F0D48035-EF9E-3141-9F63-566920E60D7C> /Users/bahk_insung/miniconda3/lib/python3.10/site-packages/torchvision/image.so
  Expected in:     <44B645FB-F027-3EE5-86D7-DBF8E2FC6264> /Users/bahk_insung/miniconda3/lib/python3.10/site-packages/torch/lib/libtorch_cpu.dylib
  warn(f"Failed to load image Python extension: {e}")


In [2]:
alexnet    = models.alexnet().to(device)
resnet18   = models.resnet18().to(device)
vgg16      = models.vgg16().to(device)
densenet   = models.densenet161().to(device)
inception  = models.inception_v3().to(device)
googlenet  = models.googlenet().to(device)
shufflenet = models.shufflenet_v2_x1_0().to(device)
mobilenet_v2 = models.mobilenet_v2().to(device)
resnext50_32x4d = models.resnext50_32x4d().to(device)
wide_resnet50_2 = models.wide_resnet50_2().to(device)
mnasnet         = models.mnasnet1_0().to(device)

/Users/bahk_insung/miniconda3/lib/python3.10/site-packages/torchvision/models/inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(
/Users/bahk_insung/miniconda3/lib/python3.10/site-packages/torchvision/models/googlenet.py:47: FutureWarning: The default weight initialization of GoogleNet will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(


# Setting datasets

In [4]:
cifar_traindataset = torchvision.datasets.CIFAR10(root='../data/', train=True, download=True, transform=transforms.ToTensor())
cifar_trainloader  = DataLoader(cifar_traindataset, batch_size=BATCH_SIZE, shuffle=True)

cifar_testdataset = torchvision.datasets.CIFAR10(root='../data/', train=False, download=True, transform=transforms.ToTensor())
cifar_testloader = DataLoader(cifar_testdataset, batch_size=BATCH_SIZE, shuffle=True)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ../data/cifar-10-python.tar.gz to ../data/
Files already downloaded and verified


# Resnet18 tuning

In [5]:
model = models.resnet18(pretrained=True)
model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)
model = model.to(device)

/Users/bahk_insung/miniconda3/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/Users/bahk_insung/miniconda3/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
total, correct = 0, 0

with torch.no_grad():
    for inputs, labels in cifar_testloader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs).to(device)
        _, predictions = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predictions == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

Accuracy of the network on the 10000 test images: 9 %


# Alexnet tuning

In [7]:
model = torchvision.models.alexnet(pretrained = True)
print(model)

num_ftrs = model.classifier[6].in_features # 4096 classes classification
model.classifier[6] = nn.Linear(num_ftrs, 10)
model = model.to(device)

/Users/bahk_insung/miniconda3/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /Users/bahk_insung/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth


  0%|          | 0.00/233M [00:00<?, ?B/s]

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [8]:
for i, (name, params) in enumerate(model.named_parameters()):
    print(i, name)

0 features.0.weight
1 features.0.bias
2 features.3.weight
3 features.3.bias
4 features.6.weight
5 features.6.bias
6 features.8.weight
7 features.8.bias
8 features.10.weight
9 features.10.bias
10 classifier.1.weight
11 classifier.1.bias
12 classifier.4.weight
13 classifier.4.bias
14 classifier.6.weight
15 classifier.6.bias


In [9]:
for i, (name, params) in enumerate(model.named_parameters()):
    params.requires_grad = False
    if i == 9:
        print('end')
        break

end


In [10]:
feature_list = [0, 3, 6, 8, 10]
classifier_list = [1, 4, 6]
for i in feature_list:
    print(model.features[i].weight.requires_grad)
    print(model.features[i].bias.requires_grad)

for j in classifier_list:
    print(model.classifier[j].weight.requires_grad)
    print(model.classifier[j].weight.requires_grad)

False
False
False
False
False
False
False
False
False
False
True
True
True
True
True
True


In [11]:
optimizer = optim.Adam(model.parameters(), lr=1e-3)
critention = nn.CrossEntropyLoss()

In [12]:
def validation_loss(dataloader):
    n = len(dataloader)
    running_loss = 0.0
    with torch.no_grad():
        model.eval()
        for data in dataloader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images)
            loss = critention(outputs, labels)
            running_loss += loss.item()

        model.train()
        return running_loss / n

In [13]:
running_loss = 0.0
test_running_loss = 0.0

for epoch in range(EPOCH + 1):
    for inputs, labels in cifar_trainloader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = critention(inputs, outputs)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        test_running_loss += validation_loss(cifar_testloader)

        if epoch % 5 == 0:
            print(f"Epoch : {epoch}, train loss : {running_loss}, test loss : {test_running_loss}")


RuntimeError: Given input size: (256x1x1). Calculated output size: (256x0x0). Output size is too small